In [ ]:
# @title # 🧜‍♀️ AutoLoRA-Merging

# @markdown Merge multipe LoRA adapters together with `Ties`, `DareLinear`, `MagnitudePrune`, `Cat`,

# @markdown `Linear`, `SVD` merging techniques.

# @markdown 🔮 Created by [@zainulabideen](https://huggingface.co/abideen).

# @markdown ---

# @markdown ### 🤗 Hugging Face Hub

MODEL_ID = "abideen/MulitLoRA-Mistral-Merging" # @param {type:"string"}
MODEL_NAME = MODEL_ID.split('/')[-1]
# username = "abideen" # @param {type:"string"}
token = "" # @param {type:"string"}
!pip install -q huggingface_hub
from huggingface_hub import create_repo, HfApi

# @markdown ---

# @markdown ### ⚡ Merging parameters

!pip install git+https://github.com/huggingface/transformers
!pip install git+https://github.com/huggingface/peft
!pip install datasets accelerate bitsandbytes


from peft import PeftConfig, PeftModel
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
import torch
import random

PEFT_MODEL_ID = "Yhyu13/dolphin-2.6-mistral-7b-dpo-laser-function-calling-lora" # @param {type:"string"}
FIRST_ADAPTER_NAME = "functioncalling" # @param {type:"string"}
SECOND_ADAPTER = "predibase/wikisql" # @param {type:"string"}
THIRD_ADAPTER = "predibase/legal" # @param {type:"string"}
device_map = {"": 0}
config = PeftConfig.from_pretrained(PEFT_MODEL_ID)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, device_map=device_map)
tokenizer = AutoTokenizer.from_pretrained(PEFT_MODEL_ID)
model.resize_token_embeddings(len(tokenizer))
model = PeftModel.from_pretrained(model, PEFT_MODEL_ID, adapter_name=FIRST_ADAPTER_NAME)
_ = model.load_adapter(SECOND_ADAPTER, adapter_name=SECOND_ADAPTER)
_ = model.load_adapter(THIRD_ADAPTER, adapter_name=THIRD_ADAPTER)

adapters = [FIRST_ADAPTER_NAME, SECOND_ADAPTER, THIRD_ADAPTER]
weights = [2.0, 0.3, 0.7]
adapter_name = "merge"
MERGING_TYPE = "magnitude_prune" # @param ["ties", "dare_linear", "magnitude_prune", "cat", "linear", "svd"]
DENSITY = 0.2 # @param {type:"number"}
if adapter_name in model.peft_config:
    model.delete_adapter(adapter_name)
model.add_weighted_adapter(adapters, weights, adapter_name, combination_type=combination_type, density=density)
model.eval()
model.set_adapter("merge")
prompt = "Table: Sports; Columns: ['Team', 'Head Coach', 'President', 'Home Ground', 'Location'] Natural Query: Who is the Head Coach of the team whose President is Mario Volarevic? SQL Query:" # @param {type:"string"}
messages = [
    {"role": "user", "content": prompt},
]
text = tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)
inputs = tokenizer(text, return_tensors="pt")  # , add_special_tokens=False)
inputs = {k: v.to("cuda") for k, v in inputs.items()}
outputs = model.generate(
    **inputs,
    max_new_tokens=256,
    do_sample=True,
    top_p=0.95,
    temperature=0.2,
    repetition_penalty=1.2,
    eos_token_id=tokenizer.eos_token_id,
)
print(tokenizer.decode(outputs[0]))
output="./out"
model.save_pretrained(output)
tokenizer.save_pretrained(output)
merge_file="./out/merge"
api = HfApi()

# Create empty repo
create_repo(
    repo_id = MODEL_ID,
    repo_type="model",
    exist_ok=True,
    token=hf_token
)

# Upload gguf files
api.upload_folder(
    folder_path=merge_file,
    repo_id=MODEL_ID,
    token=hf_token
)